<h1>Analysis of Cost / CO2 Emissions Reduction Tradeoffs</h1>

The cost / CO2 emissions reduction tradeoffs are calculated and analyzed using the Green Software Foundation API/SDKs to access the Watttime data.

In [9]:
# Libraries and Input Data
import urllib.parse
import requests
import pandas as pd

# The forecast window is fixed at 24 hours, i.e. 1440 minutes
forecast_window = '1440'
# The cost / CO2 emission reduction ratio is a 0 to 10 scale
# Test Data
cost_emphasis = 10
co2_emission = 10 - cost_emphasis
# GCP (Google Cloud Platform) normalized test data

# GCP to Azure region mapping test data.
azure_regions = ['canadacentral','canadaeast','centralus','eastus','eastus2','northcentralus','southcentralus',\
                 'westcentralus','westus','westus2','westus3',\
                 'francecentral','germanywestcentral','northeurope','norwayeast','uksouth','ukwest','westeurope',\
                 'swedencentral','australiaeast','australiacentral','australiasoutheast']

gcp_regions = ['northamerica-northeast1','northamerica-northeast2','us-east5','us-east1','us-east4','us-central1',\
               'us-south1','us-west2','us-west1','us-west4','us-west3',\
                'europe-west9','europe-west3','europe-west2','europe-north1','europe-west2','europe-west2',\
                'europe-west4','europe-north1','australia-southeast1','australia-southeast1','australia-southeast2']




### Azure and GCP Region and Location Mapping

Note: This mapping is for demonstration purposes and may not (typically will not) align with actual power sources
<pre>

</pre>
Azure&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;GCP
Region| Location | Region | Location | GCP "Low Carbon"
---|---|---|---|---
canadacentral | Toronto | northamerica-northeast2 | Toronto
canadaeast | Quebec City | northamerica-northeast1 | Montréal
centralus | Iowa | us-east5 | Columbus
eastus | Virginia | us-east1 | South Carolina
eastus2 | Virginia | us-east4 | N. Virginia
northcentralus | Illinois | us-central1 | Iowa | Y
southcentralus | Texas | us-south1 | Dallas
westus | California | us-west2 | Los Angeles
westus2 | Washington | us-west1 | Oregon | Y
westus3 | Arizona | us-west4 | Las Vegas
westcentralus | Wyoming | us-west3 | Salt Lake City
francecentral | Paris | europe-west9 | Paris | Y
germanywestcentral | Frankfurt | europe-west3 | Frankfurt
northeurope | Ireland | europe-west2 | London
norwayeast | Oslo | europe-north1 | Finland | Y
uksouth | London | europe-west2 | London
ukwest | Cardiff | europe-west2 | London
westeurope | Netherlands | europe-west4 | Netherlands
swedencentral | Gävle | europe-north1 | Finland | Y
australiaeast | New South Wales | australia-southeast1 | Sydney
australiacentral | Canberra | australia-southeast1 | Sydney
australiasoutheast | Victoria | australia-southeast2 | Melbourne

<pre>

</pre>
Coming Soon
Region| Location | Region | Location
---|---|---|---
eastus3 | Georgia | us-east1 | South Carolina
austriaeast | Vienna | europe-west3 | Frankfurt
denmarkeast | Copenhagen | europe-north1 | Finland | Y
greececentral | Athens | europe-west8 | Milan
italynorth | Milan | europe-west8 | Milan
polandcentral | Warsaw | europe-central2 | Warsaw
spaincentral | Madrid | europe-southwest1 | Madrid | Y

<pre>

</pre>
Not supported by Green Software Fundation API or Watttime
Region| Location | Region | Location
---|---|---|---
belgiumcentral | Brussels | europe-west1 | Belgium
finlandcentral | Helsinki | europe-north1 | Finland | Y
switzerlandnorth | Zürich | europe-west6 | Zürich | Y


In [10]:
# The most recent forecasted emission data with the calculated optimal marginal intensity window per region

cas_url = 'https://carbon-aware-api.azurewebsites.net'
cas_uri_path = '/emissions/forecasts/current'
regions = azure_regions
locations = []
for r in regions:
  locations.append('location={}'.format(r))
location_str = '&'.join(locations)
start_time = urllib.parse.quote('')
end_time = urllib.parse.quote('')
window_size = urllib.parse.quote(forecast_window)
# Full url is "req_url = '{}{}?{}&time={}&toTime={}&windowSize={}'.format(cas_url,cas_uri_path,location_str,
# start_time,end_time,window_size,)" however, we are not using the start and end time.
req_url = '{}{}?{}&windowSize={}'.format(
  cas_url,
  cas_uri_path,
  location_str,
  window_size,
)
print(req_url, '\n')
r = requests.get(req_url)
forecast_current_df = pd.DataFrame(r.json())
result_df = pd.DataFrame(columns=['region','location','emission_value','norm_em_value','timestamp'])
print("result_df")
display(result_df)
result_df['region'] = forecast_current_df['location']

print("forecast_data = ")                                     # Lable for testing
# Popularte the best emissions data and find max & min of the best emissions locations
max_emissions = 0
i = 0 
for row in forecast_current_df.itertuples():
  forecast_data = forecast_current_df.at[i, 'forecastData']
  print(forecast_data)                                        # Data for testing
  data = forecast_data[0]
  result_df.at[i,'location'] = data['location']
  result_df.at[i,'emission_value'] = data['value']
  result_df.at[i,'timestamp'] = data['timestamp']

  # Find the maximum and minimum of the best emisson values
  if i == 0:
    min_emissions = result_df.at[i,'emission_value']
  if max_emissions < result_df.at[i,'emission_value']:
    max_emissions = result_df.at[i,'emission_value']
  if min_emissions > result_df.at[i,'emission_value']:
    min_emissions = result_df.at[i,'emission_value']
  i = i+1

# Normalize the best emisson values
i = 0 
for row in forecast_current_df.itertuples():
  forecast_data = forecast_current_df.at[i, 'forecastData']
  data = forecast_data[0]
  #norm = (data['value'] - min_emissions)/(max_emissions - min_emissions)
  result_df.at[i,'norm_em_value'] = (data['value'] - min_emissions)/(max_emissions - min_emissions)
  i = i+1

# View Results
print(" ")
print("Normalization Range")
print("max_emissions = ", max_emissions)
print("min_emissions = ", min_emissions, "\n")

print("result_df")
display(result_df)


https://carbon-aware-api.azurewebsites.net/emissions/forecasts/current?location=canadacentral&location=canadaeast&location=centralus&location=eastus&location=eastus2&location=northcentralus&location=southcentralus&location=westcentralus&location=westus&location=westus2&location=westus3&location=francecentral&location=germanywestcentral&location=northeurope&location=norwayeast&location=uksouth&location=ukwest&location=westeurope&location=swedencentral&location=australiaeast&location=australiacentral&location=australiasoutheast&windowSize=1440 

result_df


,region,location,emission_value,norm_em_value,timestamp


forecast_data = 
[{'location': 'IESO_NORTH', 'timestamp': '2022-11-05T20:20:00+00:00', 'duration': 1440, 'value': 366.61851730789033}]
[{'location': 'HQ', 'timestamp': '2022-11-05T20:20:00+00:00', 'duration': 1440, 'value': 389.2570510512941}]
[{'location': 'MISO_MASON_CITY', 'timestamp': '2022-11-05T20:20:00+00:00', 'duration': 1440, 'value': 676.0515401489249}]
[{'location': 'PJM_ROANOKE', 'timestamp': '2022-11-05T20:20:00+00:00', 'duration': 1440, 'value': 572.5322859420859}]
[{'location': 'PJM_DC', 'timestamp': '2022-11-05T20:20:00+00:00', 'duration': 1440, 'value': 572.6228974824581}]
[{'location': 'PJM_CHICAGO', 'timestamp': '2022-11-05T20:20:00+00:00', 'duration': 1440, 'value': 559.9795104227501}]
[{'location': 'ERCOT_SANANTONIO', 'timestamp': '2022-11-05T20:20:00+00:00', 'duration': 1440, 'value': 536.0254522764502}]
[{'location': 'PACE', 'timestamp': '2022-11-05T20:20:00+00:00', 'duration': 1440, 'value': 662.4838422426435}]
[{'location': 'CAISO_NORTH', 'timestamp': '2022-11-

,region,location,emission_value,norm_em_value,timestamp
0,canadacentral,IESO_NORTH,366.618517,0.16183,2022-11-05T20:20:00+00:00
1,canadaeast,HQ,389.257051,0.217502,2022-11-05T20:20:00+00:00
2,centralus,MISO_MASON_CITY,676.05154,0.922779,2022-11-05T20:20:00+00:00
3,eastus,PJM_ROANOKE,572.532286,0.668208,2022-11-05T20:20:00+00:00
4,eastus2,PJM_DC,572.622897,0.668431,2022-11-05T20:20:00+00:00
5,northcentralus,PJM_CHICAGO,559.97951,0.637338,2022-11-05T20:20:00+00:00
6,southcentralus,ERCOT_SANANTONIO,536.025452,0.578431,2022-11-05T20:20:00+00:00
7,westcentralus,PACE,662.483842,0.889414,2022-11-05T20:20:00+00:00
8,westus,CAISO_NORTH,429.307695,0.315994,2022-11-05T20:20:00+00:00
9,westus2,GCPD,534.82611,0.575482,2022-11-05T20:20:00+00:00


In [11]:
# The most recent forecasted GCP cost estimate data per region

#<<< This cell is a work in progress >>>

In [12]:
# The optimal location to deploy the cloud application based on the cost / CO2 emission tradeoff

import random

merge_result_df = pd.DataFrame(columns=['azure_region','norm_em_value','gcp_region','norm_est_value'])
merge_result_df[['azure_region','norm_em_value']] = result_df[['region','norm_em_value']]

# Create GCP Cost Estimate test data 
norm_est=[]
i = 0 
while i < len(azure_regions):
  norm_est.append((random.randrange(0, 100))/100)
  i=i+1
#===================================

merge_result_df['gcp_region'] = gcp_regions
merge_result_df['norm_est_value'] = norm_est

#display(merge_result_df)


final_result_df = pd.DataFrame(columns=['azure_region','norm_em_value','gcp_region','norm_est_value'])
ttest = 0
ftest = 0
i = 0
temp_result = 0.0
final_result = 1.0
for i in range(len(azure_regions)):
  j = 0
  while j < len(azure_regions):
    temp_result = ((co2_emission * merge_result_df.at[i,'norm_em_value']) + (cost_emphasis * merge_result_df.at[j,'norm_est_value']))/10
    #print("co2 = ", (co2_emission * merge_result_df.at[i,'norm_em_value']), "   cost = ", (cost_emphasis * merge_result_df.at[j,'norm_est_value']))
    if temp_result < final_result:
      #print("final_result = ", final_result)
      final_result = temp_result
      final_result_df.at[0,'azure_region'] = merge_result_df.at[i,'azure_region']
      final_result_df.at[0,'norm_em_value'] = merge_result_df.at[i,'norm_em_value']
      final_result_df.at[0,'gcp_region'] = merge_result_df.at[i,'gcp_region']
      final_result_df.at[0,'norm_est_value'] = merge_result_df.at[i,'norm_est_value']
    j = j+1
  i = i+1

print(co2_emission, cost_emphasis)
print("final_result = ", final_result)
display(final_result_df)



0 10
final_result =  0.06


,azure_region,norm_em_value,gcp_region,norm_est_value
0,canadacentral,0.16183,northamerica-northeast1,0.54
